# How to use `random_grid`

In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../')
import qreative

The `random_grid` tool allows you to create random grids of `0`s and `1`s. When creating the object, you'll need to tell it how big the grid should be. For example, let's make a $4\times4$ grid.

In [2]:
grid = qreative.random_grid(5,4)

The object sets up a register of qubits, once for each position in the grid. Each is initially in state `0`.

As a quick way of determining the points that neighbour any given point, you can use the `neighbours()` method.

In [3]:
grid.neighbours( (2,2) )

[(3, 2), (1, 2), (2, 3), (2, 1)]

To read out the state of the grid we call the `get_samples()` method, which has the standard kwargs `device`, `noisy` and `shots` as explained in [the README](README.md). This runs the circuit for `shots` samples, and gets a readout of the grid for each.

Here's an example for just three samples.

In [4]:
grid_stats,grid_data = grid.get_samples(shots=3)

The first output, here called `grid_stats` is a dictionary with grid states as keys and the number of samples for which this output occurred as values.

In [5]:
for sample in grid_stats:
    print(grid_stats[sample],'shots returned the grid state\n')
    print(sample,'\n')

3 shots returned the grid state

00000
00000
00000
00000 



The second output, here called `grid_data` is a list of each individual sample.

In [6]:
for sample in grid_data:
    print(sample,'\n')

00000
00000
00000
00000 

00000
00000
00000
00000 

00000
00000
00000
00000 



Here we always got a grid full of `0`s, because nothing was done to change the grid from its initial state. So let's now make some changes.

We can use the `NOT` method to apply a NOT gate to a qubit (flips `0` to `1` and vice-versa). Let's do this to the qubit at $(0,0)$ (this is the top-left, since we use [index notation](https://en.wikipedia.org/wiki/Index_notation) and number from 0.

In [7]:
grid.NOT((0,0))

Unlike bits we can also do half a NOT: something that needs to be done twice to apply a full NOT. We can also do any other fraction. This is acheived using the `frac` kwarg, where you can specify which fraction of a NOT you'd like.

For example, here's half a NOT on the qubit at $(4,3)$ (the bottom right).

In [8]:
grid.NOT((4,3),frac=0.5)

There's also the `CNOT` method. This applies a NOT on one qubit (the target) only when another (the control) is in state `1`. When using this, the coordinates of both control and target must be supplied.

For example, let's use the `neighbours()` method to apply CNOTs with $(0,1)$ as the control, and its neighbours as targets.

In [9]:
control = (4,3)
for target in grid.neighbours( control ):
    grid.CNOT(control,target)

Here are the 10 samples of the result.

In [10]:
grid_stats,grid_data = grid.get_samples(shots=10)
def show_results (grid):
    for sample in grid_stats:
        print(grid_stats[sample],'shots returned the grid state\n')
        print(sample,'\n')
        
show_results(grid)

4 shots returned the grid state

10000
00000
00000
00000 

6 shots returned the grid state

10000
00000
00001
00011 



Here we see that position $(0,0)$ is always `1`. Position $(4,3)$ is 50/50 between `0` and `1` due to the half NOT. And when $(3,4)$ is a `1`, so too are its neighbours due to the CNOT.

Additionally, it is also possible to do fractions of the CNOT with its `frac` kwarg. In the following example, we put a `1` on the bottom-left corner and do half a CNOT between that and one of its neighbours.

In [11]:
grid.NOT( (0,3) )
grid.CNOT((0,3), (1,3), frac=0.5)

grid_stats,grid_data = grid.get_samples(shots=10)
show_results(grid)

6 shots returned the grid state

10000
00000
00000
11000 

3 shots returned the grid state

10000
00000
00001
10011 

1 shots returned the grid state

10000
00000
00001
11011 



Here we see that position $(0,3)$ is always `1` due to the NOT. Position $(1,3)$ is 50/50 between `0` and `1` due to the half CNOT.

The NOT and CNOT gates above were performed on the qubits using so-called x axes rotations. It is also possible to do them with y axes rotations, by setting the `axis` kwarg to `'y'`. This will make absolutely no difference to the results above, but it can lead to interesting effects when x and y axis rotations are combined.

For example, two x axis half NOTs combine to form a NOT. Let's do this on the $(0,0)$ qubit of a small grid.

In [12]:
grid = qreative.random_grid(2,2)

grid.NOT( (0,0), frac=0.5, axis='x' )
grid.NOT( (0,0), frac=0.5, axis='x' )
    
grid_stats,grid_data = grid.get_samples(shots=10)
show_results(grid)

10 shots returned the grid state

10
00 



The same is true for two y axis half NOTs.

In [13]:
grid = qreative.random_grid(2,2)

grid.NOT( (0,0), frac=0.5, axis='y' )
grid.NOT( (0,0), frac=0.5, axis='y' )
    
grid_stats,grid_data = grid.get_samples(shots=10)
show_results(grid)

10 shots returned the grid state

10
00 



But for one of each, the qubit at $(0,0)$ remains completely random.

In [14]:
grid = qreative.random_grid(2,2)

grid.NOT( (0,0), frac=0.5, axis='x' )
grid.NOT( (0,0), frac=0.5, axis='y' )
    
grid_stats,grid_data = grid.get_samples(shots=10)
show_results(grid)

6 shots returned the grid state

00
00 

4 shots returned the grid state

10
00 



By exploring these kinds of relationships, you can start generating random configurations that are hard for non-quantum computers to generate. For larger grids (but not too large), you might therefore need to upgrade to a better simulator. For example, `'ibmq_qasm_simulator'` is a cloud-based HPC resource that you can use with an IBMQ account.

In [15]:
grid = qreative.random_grid(5,5)

grid.NOT( (0,0), frac=0.5, )
for j in range(5):
    control = (j,j)
    grid.NOT( control, frac=(1/(j+1)) )
    for target in grid.neighbours( control ):
        grid.CNOT(control,target)

grid_stats,grid_data = grid.get_samples(shots=10,device='ibmq_qasm_simulator')    
show_results(grid)

2 shots returned the grid state

11000
10000
00000
00000
00000 

2 shots returned the grid state

00000
00000
00000
00000
00000 

1 shots returned the grid state

11000
10000
00000
00001
00011 

1 shots returned the grid state

10000
01000
00110
00100
00000 

1 shots returned the grid state

11000
10100
01110
00100
00000 

3 shots returned the grid state

01000
11100
01000
00000
00000 



If you want to use other Qiskit operations, the qubits and quantum circuit underlying this object they can be accessed via the `qr` and `qc` attributes of the object. The number of the qubit at point $(x,y)$ can be determined with the `address(x,y)` method.

If you want to use a real device, and want to specificly target a qubit on chip as a qubit in your grid, use the `coord_map` kwarg when creating the object. This should have coordinates as keys and qubit numbers as values.